In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [6]:
spark = (
    SparkSession.builder.appName('iris-hep')
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
    .config("spark.sql.catalog.spark_catalog.type", "hive")
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.local.type", "hadoop")
    .config("spark.sql.catalog.local.warehouse", "$PWD/warehouse")
    .enableHiveSupport()
    .getOrCreate()
)

In [7]:
df = spark.read.parquet("file.parquet")

In [8]:
df

DataFrame[run: int, luminosityBlock: bigint, event: bigint, MET: struct<pt:float,phi:float,sumet:float,significance:float,CovXX:float,CovXY:float,CovYY:float>, HLT: struct<IsoMu24_eta2p1:boolean,IsoMu24:boolean,IsoMu17_eta2p1_LooseIsoPFTau20:boolean>, PV: struct<npvs:int,x:float,y:float,z:float>, Muon: array<struct<pt:float,eta:float,phi:float,mass:float,charge:int,pfRelIso03_all:float,pfRelIso04_all:float,tightId:boolean,softId:boolean,dxy:float,dxyErr:float,dz:float,dzErr:float,jetIdx:int,genPartIdx:int>>, Electron: array<struct<pt:float,eta:float,phi:float,mass:float,charge:int,pfRelIso03_all:float,dxy:float,dxyErr:float,dz:float,dzErr:float,cutBasedId:boolean,pfId:boolean,jetIdx:int,genPartIdx:int>>, Photon: array<struct<pt:float,eta:float,phi:float,mass:float,charge:int,pfRelIso03_all:float,jetIdx:int,genPartIdx:int>>, Jet: array<struct<pt:float,eta:float,phi:float,mass:float,puId:boolean,btag:float>>, Tau: array<struct<pt:float,eta:float,phi:float,mass:float,charge:int,decayMode:

In [9]:
# df.createOrReplaceTempView("hep_table")

Py4JJavaError: An error occurred while calling o32.createOrReplaceTempView.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'spark_catalog': org.apache.iceberg.spark.SparkSessionCatalog
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1638)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:65)
	at org.apache.spark.sql.connector.catalog.CatalogManager.loadV2SessionCatalog(CatalogManager.scala:67)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$2(CatalogManager.scala:86)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$1(CatalogManager.scala:86)
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.sql.connector.catalog.CatalogManager.v2SessionCatalog(CatalogManager.scala:85)
	at org.apache.spark.sql.connector.catal

In [21]:
res_df = spark.sql("""

/* Generate the histogram */ WITH xyze_jets AS (SELECT event, TRANSFORM(JET, j -> CAST(STRUCT(j.btag, j.pt * COS(j.phi), j.pt * SIN(j.phi), j.pt * ((EXP(j.eta) - EXP(-j.eta)) / 2.0), SQRT(j.pt * COSH(j.eta) * j.pt * COSH(j.eta) + j.mass * j.mass)) AS STRUCT<btag: FLOAT, x: FLOAT, y: FLOAT, z: FLOAT, e: FLOAT>)) AS Jet FROM hep_table_main_restructured), tri_jets /* Compute jet triplets */ AS (SELECT event, m1, m2, m3 FROM xyze_jets LATERAL VIEW POSEXPLODE(Jet) AS idx1,m1 LATERAL VIEW POSEXPLODE(Jet) AS idx2,m2 LATERAL VIEW POSEXPLODE(Jet) AS idx3,m3 WHERE idx1 < idx2 AND idx2 < idx3), condensed_tri_jet /* Compute AddPxPyPzE3 and (partial) PxPyPzE2PtEtaPhiM for each TriJet system */ AS (SELECT event, m1, m2, m3, m1.x + m2.x + m3.x AS x, m1.y + m2.y + m3.y AS y, m1.z + m2.z + m3.z AS z, m1.e + m2.e + m3.e AS e, (m1.x + m2.x + m3.x) * (m1.x + m2.x + m3.x) AS x2, (m1.y + m2.y + m3.y) * (m1.y + m2.y + m3.y) AS y2, (m1.z + m2.z + m3.z) * (m1.z + m2.z + m3.z) AS z2, (m1.e + m2.e + m3.e) * (m1.e + m2.e + m3.e) AS e2 FROM tri_jets), singular_system /* Find the system with the lowest mass */ AS (SELECT event, MIN_BY(ARRAY_MAX(ARRAY(m1.btag, m2.btag, m3.btag)), ABS(172.5 - SQRT(e2 - x2 - y2 - z2))) AS btag FROM condensed_tri_jet GROUP BY event) SELECT CAST((CASE WHEN btag < 0 THEN -0.005 WHEN btag > 1 THEN 1.005 ELSE btag END - 0.005) / 0.01 AS LONG) * 0.01 + 0.005 AS x, COUNT(*) AS y FROM singular_system GROUP BY CAST((CASE WHEN btag < 0 THEN -0.005 WHEN btag > 1 THEN 1.005 ELSE btag END - 0.005) / 0.01 AS LONG) * 0.01 + 0.005 ORDER BY x NULLS LAST
""")


In [22]:
res_df.toPandas()

,x,y
0,-0.005,23
1,0.015,3
2,0.025,1
3,0.035,2
4,0.065,3
...,...,...
75,0.955,12
76,0.965,10
77,0.975,13
78,0.985,6
